<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/TempSeries_aula3_pd_dados_temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pandas com dados temporais

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import datetime

import yfinance as yf

Tipos e padrões diferentes e comversão de datas

In [ ]:
pd.to_datetime(['1/1/2023', np.datetime64('2023-01-01'), datetime.datetime(2023, 1, 1)])

DatetimeIndex(['2023-01-01', '2023-01-01', '2023-01-01'], dtype='datetime64[ns]', freq=None)

In [ ]:
# Padrão normalmente utilizado com datas (ANO-MES-DIA)
date_1 = pd.Series(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'])
date_2 = pd.Series(['2023-01-01 10:01:01', '2023-01-02 11:00:00',
                    '2023-01-03 12:01:01', '2023-01-04 15:01:01'])
date_3 = pd.Series(['2023-01-01-10-01-01', '2023-01-02-11-00-00',
                    '2023-01-03-12-01-01', '2023-01-04-15-01-01'])
# Padrão geralmente usado no Brasil (DIA/MES/ANO)
date_4 = pd.Series(['01/01/2023', '02/03/2023', '10/05/2023',
                    '22/12/2023', '01/05/2023'])

In [ ]:
pd.to_datetime(date_1)

0   2023-01-01
1   2023-01-02
2   2023-01-03
3   2023-01-04
dtype: datetime64[ns]

In [ ]:
pd.to_datetime(date_2)

0   2023-01-01 10:01:01
1   2023-01-02 11:00:00
2   2023-01-03 12:01:01
3   2023-01-04 15:01:01
dtype: datetime64[ns]

In [ ]:
# Format necessário para identificação do padrão desconhecido pelo pandas na conversão
pd.to_datetime(date_3, format='%Y-%m-%d-%H-%M-%S')

0   2023-01-01 10:01:01
1   2023-01-02 11:00:00
2   2023-01-03 12:01:01
3   2023-01-04 15:01:01
dtype: datetime64[ns]

In [ ]:
# Devido as informações de mes e dia trocados nos padrões, é necessário Format
series_date = pd.to_datetime(date_4, format='%d/%m/%Y')
series_date

0   2023-01-01
1   2023-03-02
2   2023-05-10
3   2023-12-22
4   2023-05-01
dtype: datetime64[ns]

Subtração de data

In [ ]:
# voltando 1 dia atrás em todas as datas
series_date - pd.Timedelta('1 day') # time.delta reprsenta 1 dia

0   2022-12-31
1   2023-03-01
2   2023-05-09
3   2023-12-21
4   2023-04-30
dtype: datetime64[ns]

In [ ]:
# subtraindo uma data específica da outra
series_date[0] - series_date[1]

Timedelta('-60 days +00:00:00')

In [ ]:
# desigualdades
series_date[series_date > '2023-04-01']

2   2023-05-10
3   2023-12-22
4   2023-05-01
dtype: datetime64[ns]

Extraindo informações de data e hora dos dados

In [ ]:
df_dates = pd.DataFrame({'date': series_date})
df_dates['year'] = df_dates['date'].dt.year
df_dates['month'] = df_dates['date'].dt.month
df_dates['day'] = df_dates['date'].dt.day
df_dates['week'] = df_dates['date'].dt.isocalendar().week
df_dates['weekday'] = df_dates['date'].dt.weekday # busca o número do dia da semana

In [ ]:
df_dates

,date,year,month,day,week,weekday
0,2023-01-01,2023,1,1,52,6
1,2023-03-02,2023,3,2,9,3
2,2023-05-10,2023,5,10,19,2
3,2023-12-22,2023,12,22,51,4
4,2023-05-01,2023,5,1,18,0


obs: o dia da semana começa a ser contado a partir da segunda feira, indo até domingo. Logo, a data da índice '0' (que foi um domingo) está na semana 52 de 2022. Outro detalhe também é que pelo padrão do dia da semana, segunda é igual a '0' indo até domingo que é o dia de índice '6', contabilizando 7 días da semana.

Aplicando na base de dados ITAUSA

In [ ]:
def get_yfinance(id_name):
  df = yf.Ticker(id_name)
  df = df.history(period='max')
  return df['Close']

id_name = 'ITSA4.SA'
serie_itausa = get_yfinance(id_name)

In [ ]:
# reseta o índice para que o índice original que eram as datas se tornem uma coluna da variável 'série itausa'
serie_itausa = serie_itausa.reset_index()

In [ ]:
# obtendo apenas as informações da data, pois as informações de hora estão zeradas
serie_itausa['Date'] = serie_itausa['Date'].dt.date

Não necessariamente um **valor nulo** conterá 'none'. As vezes um dado ausente no contexto de séries temporais pode ser um gap de tempo (se não há dado, não haverá o valor da data naquele momento.

In [ ]:
# 'df_dates' é um dataframe criando por um 'data_range()'. P parâmetro 'B' é para granularidade de business day
# ou seja, finais de semana e feriados a bolsa não opera e logo não haverão valores de fechamento
# para estes dias
df_dates = pd.DataFrame({'Date': pd.date_range(start= '2000-01-03', end = '2023-11-07', freq='B')})
df_dates['Date'] = df_dates['Date'].dt.date # Ajustando o formato da série 'df_dates'

In [ ]:
# um merge entre os dois datasets ('df_dates' e 'serie_itausa') é feito, com a parte esquerda do df_dates
# (coluna 'Date') na coluna 'Date' também da 'serie_itausa'
df_dates= df_dates.merge(serie_itausa, how='left', on ='Date')

In [ ]:
# após o merge, uma comparação para verificar valores nulos da 'serie_itausa' é feita,
# tendo apenas cerca de 3% da base itausa sendo nula
df_dates['Close'].isna().value_counts(normalize=True)

False    0.962874
True     0.037126
Name: Close, dtype: float64

Porcentagens elevadas de valores nulos, outliers e etc podem prejudicar o modelo. Mais do que 30% destes valores presentes na base já dificultam muito a captação de padrões e rendimento do modelo.

In [ ]:
# observando as datas que contêm valores nulos após o merge
df_dates[df_dates['Close'].isna()]

,Date,Close
1261,2004-11-02,NaN
1321,2005-01-25,NaN
1330,2005-02-07,NaN
1331,2005-02-08,NaN
1364,2005-03-25,NaN
...,...,...
6085,2023-05-01,NaN
6113,2023-06-08,NaN
6178,2023-09-07,NaN
6203,2023-10-12,NaN


In [ ]:
# o método 'ffill()' faz um preenchimento de um valor 'close' nulo com o valor
# imediatamente da data anterior válido de 'close'
# Existem muitos métodos de tratamento de valores ausentes. O ffill() é um dos mais simples
df_dates[ df_dates['Date'] > pd.to_datetime('2023-10-10')].ffill()

<ipython-input-71-cd021064facc>:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_dates[df_dates['Date'] > pd.to_datetime('2023-10-10')].ffill()


,Date,Close
6202,2023-10-11,9.096451
6203,2023-10-12,9.096451
6204,2023-10-13,8.887680
6205,2023-10-16,8.977154
6206,2023-10-17,8.788266
6207,2023-10-18,8.698792
6208,2023-10-19,8.748500
6209,2023-10-20,8.740000
6210,2023-10-23,8.730000
6211,2023-10-24,8.780000


Obs: esse preenchimento de dados nulos ou exclusão destes valores depende do contexto de negócio. Porém, este preenchimento de valores só pode ser feito para o treino do modelo, e não para teste, porque é um valor que não existe.

In [ ]:
serie_itausa['Date'] = pd.to_datetime(serie_itausa['Date']) # convertendo para datetime
serie_itausa.set_index('Date', inplace=True) # tornando a data índice novamente

In [ ]:
# Tendo o índice da série como sendo uma data, é possível fazer agrupamentos para diversas granularidades
# e operações nos valores que essas datas possuem. No exemplo abaixo, temos um resample semanal e a contagem
# da quantidad de linhas para essa semana. Poderiamos aplicar a média nos valores de fechamento por mês,
# e assim por diante
serie_itausa.resample('W').count()

,Close
Date,
2000-01-09,5
2000-01-16,5
2000-01-23,5
2000-01-30,5
2000-02-06,5
...,...
2023-10-15,4
2023-10-22,5
2023-10-29,5
